# Книжное дело

Компания-заказчик приобрела крупный сервис для чтения книг по подписке. 

**Цель:**

Проанализировать базу данных, содержащую информацияю о книгах, издательствах, авторах, а также пользовательские обзоры книг, и реализовать выгрузки из БД в соответствии с запросом заказчика. 
Эти данные помогут сформулировать ценностное предложение для нового продукта.

**Исходные данные:**

База данных, содержащая таблицы: `books`, `authors`, `publishers`, `ratings` и `reviews`.

## Подключение к базе

In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])

In [3]:
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

## Обзор данных

Исследуем таблицы, выведем первые строки и посчитаем количество записей. Для удобства создадим функцию для выполнение SQL-запроса:

In [4]:
def select(sql_query):
    return pd.io.sql.read_sql(sql_query, con = engine) 

### Данные о книгах

In [5]:
sql='''
SELECT *
FROM books
LIMIT 5
'''

select(sql)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [6]:
sql='''
SELECT COUNT(book_id)
FROM books
'''

select(sql)

,count
0,1000


Описание колонок таблицы `books` согласно документации:
- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя

Таблица содержит 1000 записей.

### Данные об авторах

In [7]:
sql='''
SELECT *
FROM authors
LIMIT 5
'''

select(sql)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [8]:
sql='''
SELECT COUNT(author_id)
FROM authors
'''

select(sql)

,count
0,636


Описание колонок таблицы `authors` согласно документации:
- `author_id` — идентификатор автора;
- `author` — имя автора.

Таблица содержит 636 записей.

### Данные об  издательствах

In [9]:
sql='''
SELECT *
FROM publishers
LIMIT 5
'''

select(sql)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [10]:
sql='''
SELECT COUNT(publisher_id)
FROM publishers
'''

select(sql)

,count
0,340


Описание колонок таблицы `publishers` согласно документации:
- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

Таблица содержит 340 записей.

### Данные о пользовательских оценках книг

In [11]:
sql='''
SELECT *
FROM ratings
LIMIT 5
'''

select(sql)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [12]:
sql='''
SELECT COUNT(rating_id)
FROM ratings
'''

select(sql)

,count
0,6456


Описание колонок таблицы `ratings` согласно документации:
- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

Таблица содержит 6456 записей.

### Данные о пользовательских обзорах на книги:

In [13]:
sql='''
SELECT *
FROM reviews
LIMIT 5
'''

select(sql)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [14]:
sql='''
SELECT COUNT(review_id)
FROM reviews
'''

select(sql)

,count
0,2793


Описание колонок таблицы `reviews` согласно документации:
- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

Таблица содержит 2793 записей.

## Анализ базы данных

Проанализируем базу данных и ответим на интересующие вопросы.

### Cколько книг вышло после 1 января 2000 года?

In [15]:
sql='''
SELECT COUNT(book_id)
FROM books
WHERE DATE_TRUNC('day', publication_date) > '01-01-2000'
'''

select(sql)

,count
0,819


**Вывод:** после 1 января 2000 года вышло 819 книг, что составляет 81.9% от общего объема.

### Какое количество обзоров и средняя оценка у каждой книги?

In [16]:
sql='''
SELECT b.book_id AS book_id,
       b.title AS title,
       COUNT(re.review_id) AS reviews_count,
       AVG(ra.rating) AS rating
FROM books AS b
LEFT JOIN reviews AS re ON b.book_id=re.book_id
LEFT JOIN ratings AS ra ON b.book_id=ra.book_id
GROUP BY b.book_id, b.title
--ORDER BY reviews_count DESC
'''

select(sql)

,book_id,title,reviews_count,rating
0,652,The Body in the Library (Miss Marple #3),4,4.500000
1,273,Galápagos,4,4.500000
2,51,A Tree Grows in Brooklyn,60,4.250000
3,951,Undaunted Courage: The Pioneering First Missio...,4,4.000000
4,839,The Prophet,28,4.285714
...,...,...,...,...
995,64,Alice in Wonderland,52,4.230769
996,55,A Woman of Substance (Emma Harte Saga #1),4,5.000000
997,148,Christine,21,3.428571
998,790,The Magicians' Guild (Black Magician Trilogy #1),4,3.500000


**Вывод:** мы определили количество обзоров и среднюю оценку для каждой книги. Если отсортировать полученный ркзультат по количеству обхоров, лидером будет произведение `Twilight (Twilight #1)` со 1120 обзорами.

### Какое издательство, выпустило наибольшее число книг толще 50 страниц?

In [17]:
sql='''
WITH top_p AS 
(SELECT publisher_id,
        COUNT(book_id) AS total_books
FROM books AS b
WHERE num_pages > 50
GROUP BY publisher_id
ORDER BY COUNT(book_id) DESC
LIMIT 1)

SELECT p.publisher_id,
       publisher,
       total_books
FROM publishers AS p
INNER JOIN top_p ON top_p.publisher_id=p.publisher_id
'''

select(sql)

,publisher_id,publisher,total_books
0,212,Penguin Books,42


**Вывод:** лидером по выпуску книг толще 50 страниц является издательство `Penguin Books`. 

### У какого автора самая высокая средняя оценка книг (книги с 50 и более оценками)?

In [18]:
sql='''
SELECT b.author_id AS author_id,
       a.author AS author,
       AVG(rating) AS avg_rating
FROM books b
INNER JOIN authors AS a ON b.author_id = a.author_id
INNER JOIN ratings AS r ON b.book_id=r.book_id
WHERE b.book_id IN 
(SELECT b.book_id AS book_id
FROM books AS b
LEFT JOIN ratings AS r ON b.book_id=r.book_id
GROUP BY b.book_id, b.author_id
HAVING COUNT(rating_id) >= 50)
GROUP BY b.author_id, a.author
ORDER BY avg_rating DESC
LIMIT 1
'''

select(sql)

,author_id,author,avg_rating
0,236,J.K. Rowling/Mary GrandPré,4.287097


**Вывод:** самая высокая средняя оценка книг (книги с 50 и более оценками) у автора `J.K. Rowling/Mary GrandPré`.

### Какое среднее количество обзоров от пользователей, которые поставили больше 50 оценок?

In [19]:
sql='''
-- Пользователи, которые поставили больше 50 оценок
WITH top_users AS 
(SELECT username AS user
FROM ratings
GROUP BY username
HAVING COUNT(rating_id) > 50)

SELECT ROUND(AVG(rev.re_cnt),0)
FROM 
(SELECT COUNT(review_id) AS re_cnt
FROM reviews AS r
WHERE r.username IN 
(SELECT *
 FROM top_users)
GROUP BY username) AS rev
'''
select(sql)

,round
0,24.0


**Вывод:** в среднем пользователь, поставивший больше 50 оценок, составляет 24 обзора на книги.

### Общий вывод:

Чтобы проанализировать базу данных мы использовали талблтцы с информацией о книгах, издательствах, авторах, а также пользовательские обзоры книг. 

По результату анализа были поучены следующие результаты:
1. После 1 января 2000 года вышло 819 книг, что составляет 81.9% от общего объема;
2. Для каждой книги было определено количество обзоров и средняя оценка. Если отсортировать полученный ркзультат по количеству обхоров, лидером будет произведение `Twilight (Twilight #1)` со 1120 обзорами;
3. Лидером по выпуску книг толще 50 страниц является издательство `Penguin Books`.
4. Cамая высокая средняя оценка книг (книги с 50 и более оценками) у автора `J.K. Rowling/Mary GrandPré`;
5. В среднем, пользователь, поставивший больше 50 оценок, составляет 24 обзора на книги.